In [ ]:
from functools import reduce
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
excel_files = [ os.path.join("/content/drive/MyDrive/random forest - 2024", file) for file in os.listdir("/content/drive/MyDrive/random forest - 2024") if file != "SP500_month_end_prices_2024_1927" and file.endswith(".xlsx")]



In [ ]:
    SP = pd.read_excel('/content/drive/MyDrive/random forest - 2024/SP500_month_end_prices_2024_1927.xlsx')
    data =  SP.iloc[:,1]


In [ ]:
    SP = SP.set_index('Date').resample('M').ffill().reset_index()
    SP['NextMonthDirection'] = SP['Close'].pct_change()
    SP['NextMonthDirection'] = SP['NextMonthDirection'].shift(-1)
    SP = SP.dropna()

In [ ]:
SP['NextMonthDirection'].describe()

,NextMonthDirection
count,1160.000000
mean,0.006373
std,0.053596
min,-0.299423
25%,-0.019444
50%,0.009244
75%,0.035969
max,0.391376


In [ ]:
ranges = [(-999, -0.1, 0), (-0.1, -0.05, 1), (-0.05, 0, 2), (0, 0.05, 3), (0.05, 0.1, 4), (0.1, 999, 5)]

# Create a function to classify values
def classify_value(value):
    for low, high, class_name in ranges:

        if low <= value <= high:
            return class_name
SP['NextMonthDirection'] = SP['NextMonthDirection'].apply(classify_value)

# Date = SP[['Date', 'Close']].copy().reset_index(drop = True)
# numeric_SP = pd.DataFrame(SP['NextMonthDirection'], columns = ['NextMonthDirection'])

# # Apply the `classify_value` function only to the numeric columns
# classified_numeric_SP = numeric_SP.applymap(classify_value)

# # Combine the classified numeric data with the 'date' column from the original DataFrame
#SP = Date.join(classified_numeric_SP).reset_index(drop=True)

In [ ]:
  master = []

  for i in excel_files:
      df = pd.read_excel(i)
      data =  df.iloc[:,1]
      df = df.set_index('Date').resample('M').ffill().reset_index().fillna(method='ffill').dropna()
      num_lags = 6
      exclude_columns = ['Date']
      lag_columns = [col for col in df.columns if col not in exclude_columns]
      for column in lag_columns:
          for i in range(1, num_lags + 1):
              df[f'{column}_Lag_{i}'] = df[column].shift(i)
      df = df.fillna(method='ffill').dropna()
      Date = pd.DataFrame(df['Date'], columns=['Date']).reset_index(drop=True)
      a =[]
      a.append(SP)
      a.append(df)
      df = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                              how='outer'), a)
      df = df.dropna().reset_index(drop=True)
      master.append(df)

<ipython-input-9-b4c3c5b7c507>:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.set_index('Date').resample('M').ffill().reset_index().fillna(method='ffill').dropna()
<ipython-input-9-b4c3c5b7c507>:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()
<ipython-input-9-b4c3c5b7c507>:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.set_index('Date').resample('M').ffill().reset_index().fillna(method='ffill').dropna()
<ipython-input-9-b4c3c5b7c507>:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()
<ipython-input-9-b4c3c5b7c507>:6: FutureWa

In [ ]:
Random_state =123

#Define Randome Forest Model
def RF(input,Random_state):

    X = input.drop(['NextMonthDirection', 'Date'], axis=1)  # Features
    Y = input['NextMonthDirection']  # Target variable
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    rf_classifier = RandomForestClassifier(n_estimators=10000, random_state=Random_state) # perform grid search for hyper parameter tuning to be done
    rf_classifier.fit(X_train, Y_train)
    A = rf_classifier.predict(X) # why are we predicting on everything?
    Random_state += 1
    return(A)

# Combined model using Random Forest
collective_result = []
individual_result =[]
def Combined_RF():
    randon_result = []
    a = 0
    for item in master:
        df = pd.DataFrame(RF(item,Random_state), columns= [f'Prediction_c{a+1}'])
        result_df = pd.concat([ item['Date'].reset_index(drop=True), df.reset_index(drop=True)], axis=1)
        randon_result.append(result_df)
        a += 1
    randon_result.append(SP)
    result = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                                how='outer'), randon_result)
    individual_result.append(result.dropna())
    return(RF(result.dropna(),Random_state))


# Final output as excel file
for i in range(1):
    collective_result.append(Combined_RF())

individual_result = pd.concat([pd.DataFrame(data) for data in individual_result])
individual_result = individual_result #what is the difference between both?
individual_result.to_excel("/content/drive/MyDrive/random forest - 2024/individual_result.xlsx", index=False)
collective_result = pd.concat([pd.DataFrame(data) for data in collective_result])
collective_result = collective_result
collective_result.to_excel("/content/drive/MyDrive/random forest - 2024/collective_result.xlsx", index=False)

print(collective_result.tail())
print(individual_result.tail())



     0
158  2
159  3
160  3
161  3
162  3
          Date  Prediction_c1  Prediction_c2  Prediction_c3  Prediction_c4  \
158 2024-03-31            2.0            2.0            2.0            3.0   
159 2024-04-30            3.0            3.0            3.0            3.0   
160 2024-05-31            3.0            3.0            3.0            3.0   
161 2024-06-30            3.0            3.0            3.0            3.0   
162 2024-07-31            3.0            2.0            3.0            3.0   

           Close  NextMonthDirection  
158  5254.350098                   2  
159  5035.689941                   3  
160  5277.509766                   3  
161  5460.479980                   3  
162  5463.540039                   2  
